# サーチエンジンとしての利用手順(集計)
----
検索したドキュメントをさらに集計する 方法について説明します。

## 準備

本章のコマンドを実行するための設定を行います。  

セットアップ済みの環境のうちコマンドを送る先であるCoordinating(Client) Nodeのホストアドレスを、次のセルに記入して実行し、保存してください。

In [1]:
%run rc.py
%env ES_HOST={es_host}:9200

env: ES_HOST=10.24.128.93:9200


また、データを蓄積する先のインデックスとタイプを、次のセルを実行して保存してください。  

In [2]:
%env INDEX=meteorological-data-*
%env TYPE=logs

env: INDEX=meteorological-data-*
env: TYPE=logs


## Aggregationとは

Aggregationとは、検索結果を集約・集計するための機能です。  
[02_Search](02_Search.ipynb)の「ドキュメントの検索」で説明したような検索の結果に対して、
SQLで言うと集合関数（SUM, MIN, MAX, AVGなど）や、グループ化（GROUP BY）に相当する処理を実現できます。  
また、複数のAggregationを1回のリクエストでまとめて実行したり、入れ子にして多段の計算をしたりすることも可能です。

Aggregationの種類は次の通りです。

| 種類 | 内容 | 
|------|------|
|Metrics Aggregations|複数のドキュメントが持つ数値データを集約します。<br>最大値、最小値、平均値等を求めることができます。|
|Bucket Aggregations|ドキュメントのBucket（入れ物）を生成します。<br>Bucketは大抵の場合、複数生成されます。<br>SQLで言うGROUP BY句に似た動作をします。<br>子のAggregationを指定することで、Bucketごとの集計結果を得ることもできます。|
|Pipeline Aggregations|他のAggregationの結果に対して、さらに集約処理をします。<br>SQLで言うサブクエリのような動作をします。|
|Matrix Aggregations|複数のフィールドを入力とし、結果を行列で返します。<br>相関行列を求めたりすることができます。|

## リクエストの基本構文

例えば、2015年8月の東京都の平均気温の最大値を確認するには、次の形式でリクエストを送信します。

In [3]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/_search?pretty" -d @- << EOF
{
  "size": 0,
  "query": {
    "bool": {
      "must": [
        {
          "term": {
            "location": "tokyo"
          }
        },
        {
          "range": {
            "@timestamp": {
              "gte": "2015-08-01",
              "lt": "2015-09-01",
              "time_zone": "+09:00"
            }
          }
        }
      ]
    }
  },
  "aggs" : {
    "max_temperature" : {
      "max" : {
        "field" : "temperature_max"
      }
    }
  }
}
EOF

{
  "took" : 21,
  "timed_out" : false,
  "_shards" : {
    "total" : 13,
    "successful" : 13,
    "failed" : 0
  },
  "hits" : {
    "total" : 31,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "max_temperature" : {
      "value" : 37.70000076293945
    }
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   731  100   288  100   443  10034  15435 --:--:-- --:--:-- --:--:-- 15821


この例の "**aggs**" の下の部分が、Aggregationを指定するためのJSON文です。  
"query" による条件指定と同様に、リクエストのBody部に記述します。  
"query" の検索結果に対して、"aggs"のパラメータで指定した内容で処理します。

構文は次の通りです。

|| 例での対応箇所 | 設定する内容 |
|----|----|----|
|(1)|"aggs"|Aggregationの指定が始まることを示す予約語です。<br>"aggregations"と、略さずに書くこともできます。 | 
|(2)|"temperature_max"| Aggregation名として任意の名前を付与できます。<br>例のような1件だけの場合はあまり意味がありませんが、<br>複数のAggregationを指定したとき応答を区別するためなどに役立ちます。|
|(3)|"max"| Aggregationの種別です。<br>例ではMetrics Aggregationsの1つであるmaxを指定することで、<br>最大値を計算させています。|
|(4)|"field" : "templature"|AggregationのBodyとして、集計のパラメータ等を指定します。<br>例では最大値を計算する対象として、<br>気温のフィールドを指定しています。|
|(5)|なし|"aggs"を内部に指定することで、<br>集計結果を入れ子にして計算させることができます。|
|(6)|なし|"aggs"の下に、複数のAggregationを並列に指定できます。<br>そうすることで複数の集計を同時に要求できます。<br>応答結果は指定したAggregation分だけ別々に表示され、<br>その際にAggregation名でどの指定に対応する結果かを区別できます。|

詳細については、Elasticsearch Referenceの[Aggregations](https://www.elastic.co/guide/en/elasticsearch/reference/5.x/search-aggregations.html)を参考にしてください。

なお、リクエストの例の先頭で

のように件数として0件を指定していますが、これはqueryの結果が応答に混ざるのを抑制し、集計結果だけを表示するための指定です。  

"size"は、[02_Search](02_Search.ipynb)の「検索結果の件数指定」で説明した通り、"query"で検索した結果の件数を制限するための指定です。  
この指定がない場合、例にあるリクエストを送信すると、
- "query"での検索結果（つまり2000年8月の東京都の気象データ全て）
- "aggs"での集計結果の最大値

の両方が応答に出てきてしまいます。  
今回、特に"query"に対応する検索結果は不要ですので、"size"に0件を指定することで検索結果を応答からカットしています。  

※"size"は"query"にのみ影響するため、"aggs"配下のAggregation結果が何件であっても、Aggregationの応答の数を制限しません。


## レスポンスの基本構文

Aggregationを利用して検索した場合のレスポンスは次のようになります。

"took"から"hits"までの内容は、[レスポンスの基本構文（ドキュメントID以外で検索した場合）](02_Search.ipynb#レスポンスの基本構文（ドキュメントID以外で検索した場合）)と同様です。  
"aggregations"以下について、レスポンスの例と対応させると次のような設定内容になっています。  

| 構成要素 | 例での対応箇所 | 設定される内容 |
|----|----|----|
|Aggregation名|summer_days_count|リクエストで利用したAggregation名が設定されます。<BR>1回のリクエストで複数のAggregationを行うときに結果を識別するために利用します。|
|Aggregation結果|value|Aggregation結果の値が設定されます。<BR>Aggregation種別によって、複数の結果を返す場合があります。詳細は各Aggregationの説明を参照してください。|

## Metrics Aggregations

Metrics Aggregationsは、複数のドキュメントが持つ数値データを集約するためのAggregationsです。

Metrics Aggregationsの種類は次の通りです。  
リンクになっているAggregationは、この後に説明があります。

| 名称 | Aggregation種別 | 内容 | 
|----------------------------|-----------------|------|
|[Value Count Aggregation](#Value-Count-Aggregation（個数のカウント）)|value_count|データの個数を集計します。|
|Cardinality Aggregation|cardinality|異なる値のおおよその数を集計します。|
|[Min Aggregation](#Min-/-Max-Aggregation（最小-/-最大）)|min|数値の最小値を集計します。|
|[Max Aggregation](#Min-/-Max-Aggregation（最小-/-最大）)|max|数値の最大値を集計します。|
|[Avg Aggregation](#Avg-Aggregation（平均）)|avg|数値の平均値を集計します。|
|[Sum Aggregation](#Sum-Aggregation（合計）)|sum|数値の合計値を集計します。|
|[Stats Aggregation](#Stats-Aggregation（各種集計の一括実施）)|stats|データの個数、最小値、最大値、平均値、合計値を集計します。|
|Extended Stats Aggregation|extended_stats|Stats Aggregationの結果に加えて、<br>合計値の二乗、分散、標準偏差、平均値±標準偏差を集計します。|
|[Percentiles Aggregation](#Percentiles-Aggregation（パーセンタイル）)|percentiles|データのパーセンタイル値を返します。|
|Percentile Ranks Aggregation|percentile_ranks|データのパーセンタイルランクを返します。|
|Top hits Aggregation|top_hits|関連性の高い上位何件かを返します。|
|Scripted Metric Aggregation|scripted_metric|スクリプトを利用して、独自の集約を行うことができます。|
|Geo Bounds Aggregation|geo_bounds|全ての位置座標（geo_point）を含む、長方形の座標を返します。|
|Geo Centroid Aggregation|geo_centroid|全ての位置座標（geo_point）の、中心の座標を返します。|

全Aggregationの説明については、Elasticsearch Referenceの[Metrics Aggregation](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-metrics.html)を参照してください。

### Value Count Aggregation（個数のカウント）

データの個数を集計するには、Value Count Aggregationを利用します。  
カウントしたいフィールド名をfiledパラメータとして渡します。

2015年8月の東京で、temperature_maxが「30℃以上」かつ「35℃未満」だった日を数えるを検索する(真夏日の日数を得る)コマンドは次の通りです。

In [4]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.08/_search?pretty" -d @- << EOF
{
  "size": 0,
  "query": {
    "bool": {
      "must": [
        {
          "term": {
            "location": "tokyo"
          }
        },
        {
          "range" : {
            "temperature_max" : {
              "gte" : 30,
              "lt" : 35
            }
          }
        }
      ]
    }
  },
  "aggs" : {
    "summer_days_count" : {
      "value_count" : {
        "field" : "temperature_max"
      }
    }
  }
}
EOF

{
  "took" : 4,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 12,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "summer_days_count" : {
      "value" : 12
    }
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   678  100   272  100   406  36334  54234 --:--:-- --:--:-- --:--:-- 58000


### Min / Max Aggregation（最小 / 最大）

データの最小値を集計するには、Min Aggregationを利用します。  
filedパラメータに渡したフィールドの最小値が集計されます。

最大値を集計するMax Aggregationの場合は、"min"を"max"にするだけです。

2015年1月の東京で、
- temperature_minの最小値(最も寒い日の最低気温を集計する)
- temperature_maxの最大値(最も暖かい日の最高気温を集計する)

を得るコマンドは次の通りです。

In [5]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.01/_search?pretty" -d @- << EOF
{
  "size": 0,
  "query": {
    "term": {
      "location": "tokyo"
    }
  },
  "aggs" : {
    "min_temperature" : {
      "min" : {
        "field" : "temperature_min"
      }
    },
    "max_temperature" : {
      "max" : {
        "field" : "temperature_max"
      }
    }
  }
}
EOF

{
  "took" : 3,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 29,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "max_temperature" : {
      "value" : 16.399999618530273
    },
    "min_temperature" : {
      "value" : -2.200000047683716
    }
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   617  100   354  100   263  38528  28624 --:--:-- --:--:-- --:--:-- 39333


### Avg Aggregation（平均）

Avg Aggregationは検索結果のドキュメントに含まれる数値の平均値を集計します。   
filedパラメータに渡したフィールドの平均値が集計されます。

2015年の東京都の平均気温の平均値を得るコマンドは次の通りです。

In [6]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/_search?pretty" -d @- << EOF
{
  "size": 0,
  "query": {
    "bool": {
      "must": [
        {
          "term": {
            "location": "tokyo"
          }
        },
        {
          "range": {
            "@timestamp": {
              "gte": "2015-01-01T00:00:00",
              "lt": "2016-01-01T00:00:00",
              "time_zone": "+09:00"
            }
          }
        }
      ]
    }
  },
  "aggs" : {
    "avg_temperature" : {
      "avg" : {
        "field" : "temperature_avg"
      }
    }
  }
}
EOF

{
  "took" : 5,
  "timed_out" : false,
  "_shards" : {
    "total" : 13,
    "successful" : 13,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "avg_temperature" : {
      "value" : 16.460055868052905
    }
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   750  100   289  100   461  33569  53548 --:--:-- --:--:-- --:--:-- 57625


### Sum Aggregation（合計）

データの合計を集計するには、Sum Aggregationを利用します。  
合計したいフィールド名をfiledパラメータとして渡します。

2015年1月の東京で、sunshine_duration(日照時間)を合計するコマンドは次の通りです。

In [7]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.01/_search?pretty" -d @- << EOF
{
  "size": 0,
  "query": {
    "term": {
      "location": "tokyo"
    }
  },
  "aggs" : {
    "total_sunshine_duration" : {
      "sum" : {
        "field" : "sunshine_duration"
      }
    }
  }
}
EOF

{
  "took" : 2,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 29,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "total_sunshine_duration" : {
      "value" : 167.09999953210354
    }
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   479  100   294  100   185  42682  26858 --:--:-- --:--:-- --:--:-- 49000


### Stats Aggregation（各種集計の一括実施）

ここまで、データの個数、最小、最大、平均、合計を個別に集計してきましたが、Stats Aggregationを利用すればそれらをまとめて取得できます。  
集計したいフィールド名をfiledパラメータとして渡します。

2015年1月の東京で、sunshine_duration(日照時間)を個数、最小、最大、平均、合計を取得するコマンドは次の通りです。

In [8]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.01/_search?pretty" -d @- << EOF
{
  "size": 0,
  "query": {
    "term": {
      "location": "tokyo"
    }
  },
  "aggs" : {
    "statsu_sunshine_duration" : {
      "stats" : {
        "field" : "sunshine_duration"
      }
    }
  }
}
EOF

{
  "took" : 3,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 29,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "statsu_sunshine_duration" : {
      "count" : 29,
      "min" : 0.0,
      "max" : 9.399999618530273,
      "avg" : 5.762068949382881,
      "sum" : 167.09999953210354
    }
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   586  100   398  100   188  58538  27651 --:--:-- --:--:-- --:--:-- 66333


### Percentiles Aggregation（パーセンタイル）

Percentiles Aggregationは数値のパーセンタイルを集計します。   
filedパラメータに渡したフィールドのパーセンタイルが集計されます。

2015年の東京都の最高気温について、パーセンタイルを得るコマンドは次の通りです。

In [9]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/_search?pretty" -d @- << EOF
{
  "size": 0,
  "query": {
    "term": {
      "location": "tokyo"
    }
  },
  "aggs" : {
    "percentiles_temperature" : {
      "percentiles" : {
        "field" : "temperature_max"
      }
    }
  }
}
EOF

{
  "took" : 15,
  "timed_out" : false,
  "_shards" : {
    "total" : 13,
    "successful" : 13,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "percentiles_temperature" : {
      "values" : {
        "1.0" : 5.013999986648559,
        "5.0" : 8.199999809265137,
        "25.0" : 13.481249988079071,
        "50.0" : 21.399999618530273,
        "75.0" : 27.0,
        "95.0" : 33.73000087738037,
        "99.0" : 35.385999565124514
      }
    }
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   720  100   529  100   191  27284   9851 --:--:-- --:--:-- --:--:-- 27842


## Bucket Aggregations

Bucket Aggregationsは、Metrics Aggregationsのようにフィールドの集計計算をするのではなく、  
ドキュメントのBucket（入れ物）を生成するAggregationです。  
Bucketは大抵の場合、複数生成されます。  
SQLで言うGROUP BY句に似た動作をします。

子のAggregationを指定することで、Bucketごとの集計結果を得ることもできます。

Bucket Aggregationsの種類は次の通りです。  
リンクになっているAggregationは、この後に説明があります。

| 名称 | Aggregation種別 | 内容 | 
|---------------------------|------|------|
|Children Aggregation|children|親子関係を持ったドキュメントを扱うための特殊な集計処理です。<br>親子関係の説明については、The Definitive Guideの[Parent-Child Relationship](https://www.elastic.co/guide/en/elasticsearch/guide/master/parent-child.html)を参照してください。|
|[Date Histogram Aggregation](#Date-Histogram-Aggregation（期間ごとの集計）)|date_histogram|一定期間ごとに集計します。|
|Date Range Aggregation|date_range|複数の日時の範囲ごとに区切って集計します。|
|Diversified Sampler Aggregation|diversified_sampler|集計対象を数に応じてサンプル抽出して制限します。<br>このとき指定されたフィールドについて、その値がバラバラになるよう抽出します。|
|[Filter Aggregation](#Filter-Aggregation（フィルタ）)|filter|集計対象を単一の設定でフィルタします。|
|[Filters Aggregation](#Filters-Aggregation（複数のフィルタ）)|filters|複数のフィルタごとに集計します。|
|Geo Distance Aggregation|geo_distance|中心位置の座標（緯度・経度で指定）からの距離ごとに集計します。|
|GeoHash grid Aggregation|geohash_grid|位置座標をグリッドで区切り、その範囲ごとに集計します。|
|Global Aggregation|global|queryの指定を無視して全ドキュメントを集計対象とします。|
|Histogram Aggregation|histogram|数値フィールドを一定数ごとに区切って集計します。|
|IP Range Aggregation|ip_range|IPアドレスの範囲ごとに集計します。|
|Missing Aggregation|missing|指定されたフィールドが無いドキュメントに集計対象を制限します。|
|Nested Aggregation|nested|[Nested](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/nested.html)な種別のドキュメントを扱うための特殊な集計処理です。|
|[Range Aggregation](#Range-Aggregation（値の範囲ごとの集計）)|range|数値フィールドを複数の数値範囲ごとに区切って集計します。|
|Reverse nested Aggregation|reverse_nested|Nestedなドキュメントから親のドキュメントを扱うための特殊な集計処理です。<br>Nested Aggregationの内部に定義します。|
|Sampler Aggregation|sampler|集計対象を数に応じてサンプル抽出して制限します。|
|Significant Terms Aggregation|significant_terms|指定されたフィールドについて、値の種類ごとに集計します。<br>この時、重要さのスコアを付与して返します。|
|[Terms Aggregation](#Terms-Aggregation（種類ごとの集計）)|terms|指定されたフィールドについて、値の種類ごとに集計します。|

全Aggregationの説明については、Elasticsearch Referenceの[Bucket Aggregation](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-bucket.html)を参照してください。

### 基本的な利用方法

Bucket Aggregationsは、名前の通り集計対象となるドキュメントを複数のBucket（入れ物）に仕分けします。  
(Filter, Grobal Aggregationなど単一のBucketにするような種類のものや、Nested, Reverse nested Aggregationなどの特殊用途のものもありますが、少数です）  

基本的には、Aggregationの種別と、仕分けのためのパラメータを指定します。  
次の例は、日時の期間で仕分けするDate Histogram Aggregationを用い、timestampフィールドの値で、月ごと仕分けされるようにする指定です。

Bucket Aggregationを単体で指定すると、各Bucketに入るドキュメントの数が集計されます。  
上の例であれば、月ごとの数が応答されます。

単に数を出すのでなく、Bucketごとの集計（最小、最大、合計など）を取りたいのであれば、  
Bucket Aggregationに子のAggregationを指定します。  
例えば、月ごとの平均気温を得る指定は次のようになります。

### Date Histogram Aggregation（期間ごとの集計）

区切られた期間ごとに集計するには、Date Histogram Aggregationを利用します。  
期間の設定は、年・月・日・時・分・秒のどれでも指定できます。  

区切る対象となる日時のフィールド名をfiledパラメータとして、期間の設定をintervalパラメータとして渡します。

<期間指定>の部分に  
year, quarter, month, week, day, hour, minute, second を指定すれば、それぞれ  
年、四半期、月、週、日、時、分、秒 ごとの区切りとなります。

また、 "6h" や "30m" というように数値と特定の略語を指定することで、  
"6時間ごと"、"30分ごと" のように区切ることも可能です。  

詳細は[Date Histogram Aggregation](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-bucket-datehistogram-aggregation.html)や、[time-units](https://www.elastic.co/guide/en/elasticsearch/reference/current/common-options.html#time-units) を参照してください。

2015年の東京で、temperature_maxが30℃以上だった日数を月ごとに数えるコマンドは次の通りです。

In [10]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.*/_search?pretty" -d @- << EOF
{
  "size": 0,
  "query": {
    "bool": {
      "must": [
        {
          "term": {
            "location": "tokyo"
          }
        },
        {
          "range" : {
            "temperature_max" : {
              "gte" : 30
            }
          }
        }
      ]
    }
  },
  "aggs" : {
    "hot_day_by_month" : {
      "date_histogram" : {
        "field" : "@timestamp",
        "interval":  "month"
      }
    }
  }
}
EOF

{
  "took" : 9,
  "timed_out" : false,
  "_shards" : {
    "total" : 12,
    "successful" : 12,
    "failed" : 0
  },
  "hits" : {
    "total" : 47,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "hot_day_by_month" : {
      "buckets" : [
        {
          "key_as_string" : "2015-05-01T00:00:00.000Z",
          "key" : 1430438400000,
          "doc_count" : 3
        },
        {
          "key_as_string" : "2015-06-01T00:00:00.000Z",
          "key" : 1433116800000,
          "doc_count" : 2
        },
        {
          "key_as_string" : "2015-07-01T00:00:00.000Z",
          "key" : 1435708800000,
          "doc_count" : 19
        },
        {
          "key_as_string" : "2015-08-01T00:00:00.000Z",
          "key" : 1438387200000,
          "doc_count" : 20
        },
        {
          "key_as_string" : "2015-09-01T00:00:00.000Z",
          "key" : 1441065600000,
          "doc_count" : 3
        }
      ]
    }
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1371  100   963  100   408  60680  25708 --:--:-- --:--:-- --:--:-- 64200


日時フィールドで区切りたいのでなく、数値のフィールドで区切って集計したい場合は、  
[Histogram Aggregation](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-bucket-histogram-aggregation.html)
を参照してください。  
例えば気温のフィールドを5℃ずつ区切って数えることなどが可能です。


### Filter Aggregation（フィルタ）

Aggregationで集計する対象を絞りたいときには、Filter Aggregationを利用できます。

"term"や"range"等を利用して、対象を絞り込んだ上でAggregationを適用します。  
  
"term"を利用する場合は<フィールド名>と<値>の組み合わせで集計対象の限定範囲を指定します。  
<子Aggregation>はその限定範囲内で集計を処理します。

2015年の東京で、晴れた日に限定したときのtemperature_max(最高気温)の平均を求めるコマンドは次の通りです。

In [11]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.*/_search?pretty" -d @- << EOF
{
  "size": 0,
  "query": {
    "term": {
      "location": "tokyo"
    }
  },
  "aggs" : {
    "sunny_day_temperature" : {
      "filter" : { "term": { "information_daytime": "晴" } },
      "aggs" : {
        "avg_temperature" : {
          "avg" : {
            "field" : "temperature_max"
          }
        }
      }
    }
  }
}
EOF

{
  "took" : 4,
  "timed_out" : false,
  "_shards" : {
    "total" : 12,
    "successful" : 12,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "sunny_day_temperature" : {
      "doc_count" : 41,
      "avg_temperature" : {
        "value" : 15.709756153385813
      }
    }
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   673  100   357  100   316  46297  40980 --:--:-- --:--:-- --:--:-- 51000


Filter Aggregationでの範囲限定は、queryの指定でも実現できますが、   
Filter Aggregationであれば、次のように複数個の条件を列挙することで、異なる内容の集計を同時にリクエストできる点がメリットとなります。

### Filters Aggregation（複数のフィルタ）

Filter Aggregationでは１つの条件を指定しましたが、複数を指定したい場合はFilters Aggregationが利用できます。

"term"を利用する場合は、"filters"の下に<フィールド名>と<値>の組み合わせを必要なだけ繰り返し記述します。  
<子Aggregation>の集計は、<フィールド名>と<値>の組み合わせごとに別個に処理されます。  
処理結果を区別するために<キー名>を指定します。応答には<キー名>ごとの集計結果が表示されます。

2015年の東京で、晴れ、雨、曇りの日ごとのtemperature_max(最高気温)の平均を求めるコマンドは次の通りです。

In [12]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.*/_search?pretty" -d @- << EOF
{
  "size": 0,
  "query": {
    "term": {
      "location": "tokyo"
    }
  },
  "aggs" : {
    "temperature_by_weather" : {
      "filters" : {
        "filters" : {
          "sunny"  : { "term" : { "information_daytime": "晴" }},
          "rainy"  : { "term" : { "information_daytime": "雨" }},
          "cloudy" : { "term" : { "information_daytime": "曇" }}
        }
      },      
      "aggs" : {
        "avg_temperature" : {
          "avg" : {
            "field" : "temperature_max"
          }
        }
      }
    }
  }
}
EOF

{
  "took" : 4,
  "timed_out" : false,
  "_shards" : {
    "total" : 12,
    "successful" : 12,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "temperature_by_weather" : {
      "buckets" : {
        "cloudy" : {
          "doc_count" : 8,
          "avg_temperature" : {
            "value" : 26.774999856948853
          }
        },
        "rainy" : {
          "doc_count" : 2,
          "avg_temperature" : {
            "value" : 19.949999809265137
          }
        },
        "sunny" : {
          "doc_count" : 41,
          "avg_temperature" : {
            "value" : 15.709756153385813
          }
        }
      }
    }
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1234  100   719  100   515  61531  44073 --:--:-- --:--:-- --:--:-- 65363


### Range Aggregation（値の範囲ごとの集計）

Histogram AggregationやDate Histogram Aggregationでは、一定の数値や期間で区切られた範囲を集計できますが、それぞれの範囲の大きさを個別に増減させることはできません。  
個別に増減させたいときはRange Aggragationで範囲ごとの大きさを調整できます。

区切る対象となる数値のフィールド名をfiledパラメータとして指定します。  
範囲の大きさはrangesパラメータの下で、toとfromで指定します。それを設定したい範囲ごとに繰り返します。

なお、fromと同じ値は範囲に入るよう扱われ、逆にtoと同じ値は範囲に入りません。  
また、fromかtoのどちらかは省略することも可能です。

2015年の東京で、temperature_maxが
- 15℃未満
- 15℃以上、30℃未満
- 30℃以上

である日が、3つの範囲ごとに何日ずつあったか数えるコマンドは次の通りです。

In [13]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.*/_search?pretty" -d @- << EOF
{
  "size": 0,
  "query": {
    "term": {
      "location": "tokyo"
    }
  },
  "aggs" : {
    "day_by_range" : {
      "range" : {
        "field" : "temperature_max",
        "ranges" : [
          { "to" : 15 },
          { "from" : 15, "to" :30 },
          { "from" : 30 }
        ]
      }
    }
  }
}
EOF

{
  "took" : 4,
  "timed_out" : false,
  "_shards" : {
    "total" : 12,
    "successful" : 12,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "day_by_range" : {
      "buckets" : [
        {
          "key" : "*-15.0",
          "to" : 15.0,
          "doc_count" : 106
        },
        {
          "key" : "15.0-30.0",
          "from" : 15.0,
          "to" : 30.0,
          "doc_count" : 205
        },
        {
          "key" : "30.0-*",
          "from" : 30.0,
          "doc_count" : 47
        }
      ]
    }
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   896  100   607  100   289  58590  27895 --:--:-- --:--:-- --:--:-- 60700


### Terms Aggregation（種類ごとの集計）

分類対象のフィールドが数値や期間などでなく、ある選択肢のうちどれかを選ぶようなものであった場合（例えば何かの種類の名前など）、その種類ごとに集計するにはTerms Aggregationを利用します。

設定内容としては、分類したいフィールド名をfiledパラメータとして指定するのみです。

2015年の東京で、information_daytime(日中の天気)ごとの日数を数えるコマンドは次の通りです。

In [14]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.*/_search?pretty" -d @- << EOF
{
  "size": 0,
  "query": {
    "term": {
      "location": "tokyo"
    }
  },
  "aggs" : {
    "day_by_info" : {
      "terms" : {
        "field" : "information_daytime"
      }
    }
  }
}
EOF

{
  "took" : 21,
  "timed_out" : false,
  "_shards" : {
    "total" : 12,
    "successful" : 12,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "day_by_info" : {
      "doc_count_error_upper_bound" : 0,
      "sum_other_doc_count" : 115,
      "buckets" : [
        {
          "key" : "晴",
          "doc_count" : 41
        },
        {
          "key" : "一時曇",
          "doc_count" : 30
        },
        {
          "key" : "一時雨",
          "doc_count" : 22
        },
        {
          "key" : "時々曇",
          "doc_count" : 21
        },
        {
          "key" : "一時晴",
          "doc_count" : 16
        },
        {
          "key" : "後曇",
          "doc_count" : 15
        },
        {
          "key" : "時々晴",
          "doc_count" : 11
        },
        {
          "key" : "後一時雨",
          "doc_count" : 9
        },
        {
          "key" : "後薄曇",
          "doc_count" : 9
        },
        {
  

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1315  100  1138  100   177  36928   5743 --:--:-- --:--:-- --:--:-- 37933


## Pipeline Aggregations


Pipeline Aggregationsは、他のAggregationの結果に対して、さらに集約処理をするAggregationsです。
SQLで言うサブクエリのような動作をします。

Pipeline Aggregationsの種類は次の通りです。  
リンクになっているAggregationは、この後に説明があります。

| 名称 | Aggregation種別 | 内容 | 
|-----------------------------|-----------------|------|
|[Min Bucket Aggregation](#Min-/-Max-Bucket-Aggregation（最小-/-最大）)|min_bucket|集計結果の最小値を集計します。|
|[Max Bucket Aggregation](#Min-/-Max-Bucket-Aggregation（最小-/-最大）)|max_bucket|集計結果の最大値を集計します。|
|[Avg Bucket Aggregation](#Avg-Bucket-Aggregation（平均）)|avg_bucket|集計結果の平均値を集計します。|
|[Sum Bucket Aggregation](#Sum-Bucket-Aggregation（合計）)|sum_bucket|集計結果の合計値を集計します。|
|[Derivative Aggregation](#Derivative-Aggregation（差分）)|derivative|時系列に並んだ集計結果に対し、1つ前のデータとの差分を集計します。|
|Serial Differencing Aggregation|serial_diff|時系列に並んだ集計結果に対し、いくつか前のデータとの差分を返します。|
|[Stats Bucket Aggregation](#Stats-Bucket-Aggregation（各種集計の一括実施）)|stats_bucket|集計結果の個数、最小値、最大値、平均値、合計値を集計します。|
|Extended Stats Bucket Aggregation|extended_stats_bucket|Stats Bucket　Aggregationの結果に加えて、<br>合計値の二乗、分散、標準偏差、平均値±標準偏差を集計します。|
|[Percentiles Bucket Aggregation](#Percentiles-Bucket-Aggregation（パーセンタイル）)|percentiles_bucket|集計結果のパーセンタイル値を返します。|
|[Moving Average Aggregation](#Moving-Average-Aggregation（移動平均）)|moving_avg|時系列に並んだ集計結果に対し、移動平均を返します。|
|Cumulative Sum Aggregation|cumulative_sum|集計結果の累積和を返します。|
|Bucket Script Aggregation|bucket_script|集計結果に対して、スクリプトを適用した結果を返します。|
|Bucket Selector Aggregation|bucket_selector|集計結果に対して、スクリプトに記述した条件を満たす結果を返します。|

全Aggregationの説明については、Elasticsearch Referenceの[Pipeline Aggregation](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-pipeline.html)を参照してください。

### 基本的な利用方法

Pipeline Aggregationsは、他のAggregationによる集計結果を集計します。  
集計対象は、Aggregationの親子関係等で勝手に決まるのではなく、"**buckets_path**"という専用のパラメータに対象Aggregationの名前を記述することで指定します。

次の例は、月ごとの平均気温を集計し、その中からさらに最も高いものを集計するリクエストです。  
(3)が他の集計結果から最大のものを得るというPipeline Aggregationsを指定している部分です。

(1)と(2)は、前の章までで説明しているBucket AggregationsとMetrics Aggregationsの組み合わせです。  

- (1)のData Histgram Aggregationで月ごとのBucketを生成
- (2)のAvg Aggregationで、(1)で生成されたBucketごとに平均値を集計

と、ここまでで「月ごと」の「平均気温」が集計されています。

(3)の"max_bucket"が、Max Bucket AggregationというPipeline Aggregationsの1つを指定している部分です。  
※名前に"Bucket"と入っていますが、BucketでなくPipeline Aggregationsの一種です。  

"buckets_path"パラメータは集計の対象を指定します。基本的には、

と、起点になるAggregationから順に子のAggregation名を記述し、">"で区切ります。  
起点のAggregationは、Pipeline Aggregationsの兄弟関係にある物になります。  
今回の場合であれば、(3)と兄弟関係にある(1)の"temperature_per_month"が起点になります。

よって、


と記述することで、(1),(2)の出力結果が集計対象であることを指定しています。  
Max Bucket Aggregationは、集計対象のうち最大のものを取得しますので、  
この例の場合は、「月ごと」の「平均気温」のうち「最大のもの」を返します。

"buckets_path"等の指定方法の詳細については、Elasticsearch Referenceの[Pipeline Aggregation](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-pipeline.html)を参照してください。

### Min / Max Bucket Aggregation（最小 / 最大）

集計データに対する最小値を集計するには、Min Bucket Aggregationを利用します。  
指定したパス名の集計結果の最小値が集計されます。

最大値を集計するMax Bucket Aggregationの場合は、"min_bucket"を"max_bucket"にするだけです。

2015年の東京で、平均気温がもっとも高い月を求めるケースを考えます。この場合、
- Date Histogram Aggregationで各月の平均気温を求める。
- 各月の平均気温のうち、Max Bucket Aggregationでもっとも平均気温が高い月を求める。

というクエリを実行します。

この結果を得るコマンドは次の通りです。

In [15]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.*/_search?pretty" -d @- << EOF
{
  "size": 0,
  "query": {
    "term": {
      "location": "tokyo"
    }
  },
  "aggs" : {
    "temperature_per_month" : {
      "date_histogram" : {
        "field" : "@timestamp",
        "interval" : "month"
      },
      "aggs": {
        "avg_temperature": {
          "avg": {
            "field": "temperature_avg"
          }
        }
      }            
    },
    "max_temperature_month" : {
      "max_bucket" : {
        "buckets_path" : "temperature_per_month>avg_temperature"
      }
    }
  }
}
EOF

{
  "took" : 6,
  "timed_out" : false,
  "_shards" : {
    "total" : 12,
    "successful" : 12,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "temperature_per_month" : {
      "buckets" : [
        {
          "key_as_string" : "2015-01-01T00:00:00.000Z",
          "key" : 1420070400000,
          "doc_count" : 29,
          "avg_temperature" : {
            "value" : 5.758620689655173
          }
        },
        {
          "key_as_string" : "2015-02-01T00:00:00.000Z",
          "key" : 1422748800000,
          "doc_count" : 28,
          "avg_temperature" : {
            "value" : 5.717857105391366
          }
        },
        {
          "key_as_string" : "2015-03-01T00:00:00.000Z",
          "key" : 1425168000000,
          "doc_count" : 31,
          "avg_temperature" : {
            "value" : 10.251612970905919
          }
        },
        {
          "key_as_string" : "2015-04-01T00:00:00.000Z",

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3580  100  3095  100   485   264k  42484 --:--:-- --:--:-- --:--:--  274k


### Avg Bucket Aggregation（平均）

集計データに対する平均値を集計するには、Avg Bucket Aggregationを利用します。  
指定したパス名の集計結果の平均値が集計されます。

2015年の東京で、毎月の最小気温の平均値を求めるケースを考えます。この場合、
- Date Histogram Aggregationで各月の気温の最小値を求める。
- 各月の平均気温の最小値に対して、Avg Bucket Aggregationで毎月の最小気温の平均値を求める。

というクエリを実行します。

この結果を得るコマンドは次の通りです。

In [16]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.*/_search?pretty" -d @- << EOF
{
  "size": 0,
  "query": {
    "term": {
      "location": "tokyo"
    }
  },
  "aggs" : {
    "temperature_per_month" : {
      "date_histogram" : {
        "field" : "@timestamp",
        "interval" : "month"
      },
      "aggs": {
        "min_temperature": {
          "min": {
            "field": "temperature_min"
          }
        }
      }            
    },
    "avg_temperature_month" : {
      "avg_bucket" : {
        "buckets_path" : "temperature_per_month>min_temperature"
      }
    }
  }
}
EOF

{
  "took" : 4,
  "timed_out" : false,
  "_shards" : {
    "total" : 12,
    "successful" : 12,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "temperature_per_month" : {
      "buckets" : [
        {
          "key_as_string" : "2015-01-01T00:00:00.000Z",
          "key" : 1420070400000,
          "doc_count" : 29,
          "min_temperature" : {
            "value" : -2.200000047683716
          }
        },
        {
          "key_as_string" : "2015-02-01T00:00:00.000Z",
          "key" : 1422748800000,
          "doc_count" : 28,
          "min_temperature" : {
            "value" : -2.4000000953674316
          }
        },
        {
          "key_as_string" : "2015-03-01T00:00:00.000Z",
          "key" : 1425168000000,
          "doc_count" : 31,
          "min_temperature" : {
            "value" : -0.4000000059604645
          }
        },
        {
          "key_as_string" : "2015-04-01T00:00:00.00

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3522  100  3037  100   485   337k  55176 --:--:-- --:--:-- --:--:--  370k


### Sum Bucket Aggregation（合計）

集計データに対する合計値を集計するには、Sum Bucket Aggregationを利用します。  
指定したパス名の集計結果の合計値が集計されます。

2015年の東京で、毎月の日照時間の平均を全ての月で求め、それらを合計するコマンドは次の通りです。

In [17]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.*/_search?pretty" -d @- << EOF
{
  "size": 0,
  "query": {
    "term": {
      "location": "tokyo"
    }
  },
  "aggs" : {
    "sunshine_per_month" : {
      "date_histogram" : {
        "field" : "@timestamp",
        "interval" : "month"
      },
      "aggs": {
        "avg_sunshine": {
          "avg": {
            "field": "sunshine_duration"
          }
        }
      }            
    },
    "sum_sunshine_month" : {
      "sum_bucket" : {
        "buckets_path" : "sunshine_per_month>avg_sunshine"
      }
    }
  }
}
EOF

{
  "took" : 3,
  "timed_out" : false,
  "_shards" : {
    "total" : 12,
    "successful" : 12,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "sunshine_per_month" : {
      "buckets" : [
        {
          "key_as_string" : "2015-01-01T00:00:00.000Z",
          "key" : 1420070400000,
          "doc_count" : 29,
          "avg_sunshine" : {
            "value" : 5.762068949382881
          }
        },
        {
          "key_as_string" : "2015-02-01T00:00:00.000Z",
          "key" : 1422748800000,
          "doc_count" : 28,
          "avg_sunshine" : {
            "value" : 5.960714265704155
          }
        },
        {
          "key_as_string" : "2015-03-01T00:00:00.000Z",
          "key" : 1425168000000,
          "doc_count" : 31,
          "avg_sunshine" : {
            "value" : 6.2645161382613646
          }
        },
        {
          "key_as_string" : "2015-04-01T00:00:00.000Z",
          "

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3457  100  2985  100   472   377k  61068 --:--:-- --:--:-- --:--:--  416k


### Derivative Aggregation（差分）

時系列に並んだ集計結果に対し、1つ前のデータとの差分を集計するには、Derivative Aggregationを利用します。  
Derivative Aggregationの親階層は、Date Histogram AggregationもしくはHistogram Aggregationである必要があります。
  
指定したパス名の集計結果の差分が集計されます。

2015年の東京で、毎月の最小気温の前月との差分を得るコマンドは次の通りです。

In [18]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.*/_search?pretty" -d @- << EOF
{
  "size": 0,
  "query": {
    "term": {
      "location": "tokyo"
    }
  },
  "aggs" : {
    "temperature_per_month" : {
      "date_histogram" : {
        "field" : "@timestamp",
        "interval" : "month"
      },
      "aggs": {
        "temperature": {
          "min": {
            "field": "temperature_min"
          }
        },
        "derivative_temperature_month" : {
          "derivative" : {
            "buckets_path" : "temperature"
          }
        }
      }
    }
  }
}
EOF

{
  "took" : 4,
  "timed_out" : false,
  "_shards" : {
    "total" : 12,
    "successful" : 12,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "temperature_per_month" : {
      "buckets" : [
        {
          "key_as_string" : "2015-01-01T00:00:00.000Z",
          "key" : 1420070400000,
          "doc_count" : 29,
          "temperature" : {
            "value" : -2.200000047683716
          }
        },
        {
          "key_as_string" : "2015-02-01T00:00:00.000Z",
          "key" : 1422748800000,
          "doc_count" : 28,
          "temperature" : {
            "value" : -2.4000000953674316
          },
          "derivative_temperature_month" : {
            "value" : -0.20000004768371582
          }
        },
        {
          "key_as_string" : "2015-03-01T00:00:00.000Z",
          "key" : 1425168000000,
          "doc_count" : 31,
          "temperature" : {
            "value" : -0.400000005960

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4475  100  4005  100   470   439k  52850 --:--:-- --:--:-- --:--:--  488k


### Stats Bucket Aggregation（各種集計の一括実施）

集計データの個数、最小、最大、平均、合計を一括して取得するには、Stats Bucket Aggregationを利用します。

指定したパス名の集計結果に対し、各種の集計が一括実施されます。

2015年の東京で、月ごとの平均気温の個数、最小、最大、平均、合計を一括して取得するコマンドは次の通りです。

In [19]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.*/_search?pretty" -d @- << EOF
{
  "size": 0,
  "query": {
    "term": {
      "location": "tokyo"
    }
  },
  "aggs" : {
    "temperature_per_month" : {
      "date_histogram" : {
        "field" : "@timestamp",
        "interval" : "month"
      },
      "aggs": {
        "avg_temperature": {
          "avg": {
            "field": "temperature_avg"
          }
        }
      }            
    },
    "stats_temperature_month" : {
      "stats_bucket" : {
        "buckets_path" : "temperature_per_month>avg_temperature"
      }
    }
  }
}
EOF

{
  "took" : 3,
  "timed_out" : false,
  "_shards" : {
    "total" : 12,
    "successful" : 12,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "temperature_per_month" : {
      "buckets" : [
        {
          "key_as_string" : "2015-01-01T00:00:00.000Z",
          "key" : 1420070400000,
          "doc_count" : 29,
          "avg_temperature" : {
            "value" : 5.758620689655173
          }
        },
        {
          "key_as_string" : "2015-02-01T00:00:00.000Z",
          "key" : 1422748800000,
          "doc_count" : 28,
          "avg_temperature" : {
            "value" : 5.717857105391366
          }
        },
        {
          "key_as_string" : "2015-03-01T00:00:00.000Z",
          "key" : 1425168000000,
          "doc_count" : 31,
          "avg_temperature" : {
            "value" : 10.251612970905919
          }
        },
        {
          "key_as_string" : "2015-04-01T00:00:00.000Z",

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3644  100  3155  100   489   442k  70208 --:--:-- --:--:-- --:--:--  513k


### Percentile Bucket Aggregation（パーセンタイル）

集計データに対するパーセンタイルを集計するには、Percentile Bucket Aggregationを利用します。  

指定したパス名の集計結果のパーセンタイルが集計されます。

2015年の日ごとの最高気温のパーセンタイルを得るコマンドは次の通りです。

In [20]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.*/_search?pretty" -d @- << EOF
{
  "size": 0,
  "aggs" : {
    "temperature_per_day" : {
      "date_histogram" : {
        "field" : "@timestamp",
        "interval" : "day"
      },
      "aggs": {
        "temperature": {
          "max": {
            "field": "temperature_max"
          }
        }
      }            
    },
    "percentiles_temperature_day" : {
      "percentiles_bucket" : {
        "buckets_path" : "temperature_per_day>temperature"
      }
    }
  }
}
EOF

{
  "took" : 13,
  "timed_out" : false,
  "_shards" : {
    "total" : 12,
    "successful" : 12,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "temperature_per_day" : {
      "buckets" : [
        {
          "key_as_string" : "2015-01-01T00:00:00.000Z",
          "key" : 1420070400000,
          "doc_count" : 1,
          "temperature" : {
            "value" : 8.199999809265137
          }
        },
        {
          "key_as_string" : "2015-01-02T00:00:00.000Z",
          "key" : 1420156800000,
          "doc_count" : 1,
          "temperature" : {
            "value" : 7.900000095367432
          }
        },
        {
          "key_as_string" : "2015-01-03T00:00:00.000Z",
          "key" : 1420243200000,
          "doc_count" : 1,
          "temperature" : {
            "value" : 8.899999618530273
          }
        },
        {
          "key_as_string" : "2015-01-04T00:00:00.000Z",
          "key" 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 79361  100 78935  100   426  3358k  18562 --:--:-- --:--:-- --:--:-- 3503k


### Moving Average Aggregation（移動平均）

時系列に並んだ集計結果に対し、移動平均を集計するには、Moving Average Aggregationを利用します。  
Moving Average Aggregationの親階層は、Date Histogram AggregationもしくはHistogram Aggregationである必要があります。

指定したパス名の移動平均が集計されます。windowにはウィンドウのサイズを指定します。  
例えばパス名に指定したのが日ごとのデータであり、ウィンドウのサイズに10を指定すれば、10日間の平均値が、期間を1日ずつ後ろにずらして連続して出力されます。

他にもパラメータはありますが、詳細はElasticsearch Referenceの[Moving Average Aggregation](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-pipeline-movavg-aggregation.html)を参照してください。

日ごとの最高気温の最大値について、10日間の移動平均を得るコマンドは次の通りです。

In [21]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.*/_search?pretty" -d @- << EOF
{
  "size": 0,
  "aggs" : {
    "temperature_per_day" : {
      "date_histogram" : {
        "field" : "@timestamp",
        "interval" : "day"
      },
      "aggs": {
        "temperature": {
          "max": {
            "field": "temperature_max"
          }
        },
        "10day_moving_avg" : {
          "moving_avg" : {
            "buckets_path" : "temperature",
            "window" : 10
          }
        }
      }            
    }
  }
}
EOF

{
  "took" : 13,
  "timed_out" : false,
  "_shards" : {
    "total" : 12,
    "successful" : 12,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "temperature_per_day" : {
      "buckets" : [
        {
          "key_as_string" : "2015-01-01T00:00:00.000Z",
          "key" : 1420070400000,
          "doc_count" : 1,
          "temperature" : {
            "value" : 8.199999809265137
          }
        },
        {
          "key_as_string" : "2015-01-02T00:00:00.000Z",
          "key" : 1420156800000,
          "doc_count" : 1,
          "temperature" : {
            "value" : 7.900000095367432
          },
          "10day_moving_avg" : {
            "value" : 8.199999809265137
          }
        },
        {
          "key_as_string" : "2015-01-03T00:00:00.000Z",
          "key" : 1420243200000,
          "doc_count" : 1,
          "temperature" : {
            "value" : 8.899999618530273
          },
      

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  107k  100  106k  100   433  4903k  19892 --:--:-- --:--:-- --:--:-- 5083k


## Matrix Aggregations

Matrix Aggregationsは、複数のフィールドを元に行列形式の集計結果を返すようなAggregationsです。

Matrix Aggregationsの種類は次の通りです。

| 名称 | Aggregation種別 | 内容 | 
|---------------------------|-----------------|------|
|[Matrix Stats](#Matrix-Stats)|matrix_stats|複数のフィールドを指定して、平均値、分散、相関行列などを返します。|

詳細については、Elasticsearch Referenceの[Matrix Aggregations](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-matrix.html)を参照してください。

### Matrix Stats

Matrix Statsは複数のフィールドを指定して、次の統計情報を返します。

| データの種類 | レスポンスのフィールド名 | 内容 |
|--------------|----------|------|
|データ数|count|指定したフィールドのデータ数|
|平均値|mean|指定したフィールドの平均値|
|分散|variance|指定したフィールドの分散|
|歪度|skewness|指定したフィールドの歪度|
|尖度|kurtosis|指定したフィールドの尖度|
|共分散|covariance|指定したフィールド間の共分散|
|相関係数|correlation|指定したフィールド間の相関係数|

aggregation名にmatrix_statsを指定し、複数のフィールド名を指定します。

フィールド名は3個以上指定することもできます。

詳細は、Elasticsearch Referenceの[Matrix Stats](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-matrix-stats-aggregation.html)を参照してください。

2015年の東京都の、1日の平均気温(temperature_avg)と平均湿度(humidity_avg)のMatrix Statsを確認するには、次の形式でリクエストを送信します。

In [22]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.*/_search?pretty" -d @- << EOF
{
  "query": {
    "term" : { "location" : "tokyo" } 
  },
  "aggs": {
    "agg_name": {
      "matrix_stats": {
        "fields": ["temperature_avg", "humidity_avg"]
      }
    }
  }
}
EOF

{
  "took" : 9,
  "timed_out" : false,
  "_shards" : {
    "total" : 12,
    "successful" : 12,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 0.017391743,
    "hits" : [
      {
        "_index" : "meteorological-data-2015.02",
        "_type" : "logs",
        "_id" : "tokyo_15-02-01",
        "_score" : 0.017391743,
        "_source" : {
          "date" : "15-02-01",
          "wind_speed_avg" : "5.5",
          "snowfall" : null,
          "temperature_max" : "8.6",
          "precipitation_day" : null,
          "wind_speed_max" : "10.4",
          "sunshine_duration" : "9.6",
          "temperature_avg" : "4.0",
          "temperature_min" : "0.7",
          "@version" : "1",
          "wind_direction_max_moment" : "北西",
          "information_night" : "晴",
          "precipitation_max_10min" : null,
          "precipitation_max_hour" : null,
          "wind_direction" : "北西",
          "information_daytime" : "晴",
          "@timestamp" : "2015-02-01T00

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12664  100 12489  100   175   804k  11540 --:--:-- --:--:-- --:--:--  813k


## 実行クエリ解析

クエリを解析するためにProfile APIを使用します。  

Profile APIは、実行クエリについて詳細情報を提供します。  
クエリがどのように実行されるかをユーザーに示し、特定のクエリが遅い理由を理解し、改善するために使用します。

例として、Profile APIのコマンドは次の通りです。

In [23]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/_search?pretty" -d @- << EOF
{
  "profile": true,
  "size": 0,
  "query": {
    "bool": {
      "must": [
        {
          "term": {
            "location": "tokyo"
          }
        },
        {
          "range": {
            "@timestamp": {
              "gte": "2015-01-01T00:00:00",
              "lt": "2016-01-01T00:00:00",
              "time_zone": "+09:00"
            }
          }
        }
      ]
    }
  },
  "aggs" : {
    "avg_temperature" : {
      "avg" : {
        "field" : "temperature_avg"
      }
    }
  }
}
EOF

{
  "took" : 5,
  "timed_out" : false,
  "_shards" : {
    "total" : 13,
    "successful" : 13,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "avg_temperature" : {
      "value" : 16.460055868052905
    }
  },
  "profile" : {
    "shards" : [
      {
        "id" : "[dLCXd9QZQUCno6v6ZfJkyA][meteorological-data-2015.05][0]",
        "searches" : [
          {
            "query" : [
              {
                "type" : "BooleanQuery",
                "description" : "+location:tokyo +@timestamp:[-9223372036854775808 TO 9223372036854775807]",
                "time" : "0.5378580000ms",
                "breakdown" : {
                  "score" : 0,
                  "build_scorer_count" : 2,
                  "match_count" : 0,
                  "create_weight" : 57814,
                  "next_doc" : 252709,
                  "match" : 0,
                  "create_weight_count" : 1,
                  "next_do

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 50393  100 49914  100   479  3631k  35687 --:--:-- --:--:-- --:--:-- 3749k


"profile"配下の"shards"配下にある"aggregations"に、実行したAggregationの情報があります。

Profile APIの詳細は、Elasticsearch Referenceの[Profile API](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/search-profile.html)を参照してください。